# INF-395 Redes Neuronales Artificiales II-2018

## Tarea 2 - Redes Convolucionales y sus aplicaciones


26/11/2018

* Rodrigo González Smith 201303026-2
* Ignacio Valenzuela Albornoz

## Introducción

In [35]:
import numpy as np   
import os   
import keras
import matplotlib.pyplot as plt
import math
from keras.datasets import cifar10
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.optimizers import SGD, rmsprop
from keras.callbacks import LearningRateScheduler
from keras.models import model_from_json
import os
import h5py

%matplotlib inline

## 1. Convolutional Neural Networks (CNN) en CIFAR

### 1.a)

In [49]:
label_names = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']  
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

Separamos el set de train para tener uno de validación también:

In [50]:
n_train = len(x_train)
x_val = x_train[int(0.8*n_train):n_train]
y_val = y_train[int(0.8*n_train):n_train]
x_train = x_train[:int(0.8*n_train)]
y_train = y_train[:int(0.8*n_train)]

### 1.b)

In [51]:
#x_train = x_train.transpose([0, 3, 1, 2])
#x_val = x_val.transpose([0, 3, 1, 2])
#x_test = x_test.transpose([0, 3, 1, 2])

x_train_norm = x_train/255
x_val_norm = x_val/255
x_test_norm = x_test/255

num_classes = len(label_names)
y_train = keras.utils.to_categorical(y_train, num_classes)
y_val = keras.utils.to_categorical(y_val, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

### 1.c)

In [55]:
model_1_c = Sequential()
model_1_c.add(Conv2D(64, (3, 3), padding='same', input_shape=x_train.shape[1:]))
model_1_c.add(Activation('relu'))
model_1_c.add(MaxPooling2D(pool_size=(2, 2)))
model_1_c.add(Conv2D(64, (3, 3), padding='same'))
model_1_c.add(Activation('relu'))
model_1_c.add(MaxPooling2D(pool_size=(2, 2)))
model_1_c.add(Flatten())
model_1_c.add(Dense(512))
model_1_c.add(Activation('relu'))
model_1_c.add(Dense(10))
model_1_c.add(Activation('softmax'))
model_1_c.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_18 (Conv2D)           (None, 32, 32, 64)        1792      
_________________________________________________________________
activation_28 (Activation)   (None, 32, 32, 64)        0         
_________________________________________________________________
max_pooling2d_16 (MaxPooling (None, 16, 16, 64)        0         
_________________________________________________________________
conv2d_19 (Conv2D)           (None, 16, 16, 64)        36928     
_________________________________________________________________
activation_29 (Activation)   (None, 16, 16, 64)        0         
_________________________________________________________________
max_pooling2d_17 (MaxPooling (None, 8, 8, 64)          0         
_________________________________________________________________
flatten_7 (Flatten)          (None, 4096)              0         
__________

### 1.d)

In [56]:
def step_decay(epoch):
    initial_lrate = 0.001
    lrate = initial_lrate * math.pow(0.5, math.floor((epoch)/10))
    lrate = max(lrate,0.00001)
    return lrate
opt = SGD(lr=0.0, momentum=0.9, decay=0.0)
lrate = LearningRateScheduler(step_decay)
batch_size = 1000
epochs = 25
model_1_c.compile(loss="categorical_crossentropy" , optimizer=opt )
model_1_c.fit(x_train_norm, y_train,batch_size=batch_size,epochs=epochs, validation_data=(x_val_norm,y_val), shuffle=True, callbacks=[lrate])

Train on 40000 samples, validate on 10000 samples
Epoch 1/25
40000/40000 [==============================] - 100s 2ms/step - loss: 2.3012 - val_loss: 2.2871
Epoch 2/25
40000/40000 [==============================] - 100s 2ms/step - loss: 2.2775 - val_loss: 2.2660
Epoch 3/25
40000/40000 [==============================] - 101s 3ms/step - loss: 2.2549 - val_loss: 2.2397
Epoch 4/25
40000/40000 [==============================] - 100s 3ms/step - loss: 2.2241 - val_loss: 2.2028
Epoch 5/25
40000/40000 [==============================] - 100s 3ms/step - loss: 2.1821 - val_loss: 2.1570
Epoch 6/25
40000/40000 [==============================] - 100s 3ms/step - loss: 2.1322 - val_loss: 2.1047
Epoch 7/25
40000/40000 [==============================] - 101s 3ms/step - loss: 2.0823 - val_loss: 2.0597
Epoch 8/25
40000/40000 [==============================] - 100s 3ms/step - loss: 2.0363 - val_loss: 2.0168
Epoch 9/25
40000/40000 [==============================] - 100s 3ms/step - loss: 1.9973 - val_loss: 1.9

NameError: name 'model_2_b_json' is not defined

In [58]:
model_1_c_json = model_1_c.to_json()
with open("modelos/model_1_c.json", "w") as json_file:
    json_file.write(model_1_c_json)
model_1_c.save_weights("modelos/model_1_c.h5")

### 1.e)